<a href="https://colab.research.google.com/github/sefeoglu/AE_Parseval_Network/blob/develop/src/notebooks/Grid_SearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import gzip
import pickle
import numpy as np
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from wresnet import WideResidualNetwork
import tensorflow
import cv2
import json

In [15]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    new_data_X = []
    Y_data = []
    for row in data:
        new_data_X.append(cv2.resize(row['crop'], (32,32)))
        Y_data.append(row['label'])
    new_data_X = np.array(new_data_X)

    return new_data_X, Y_data

In [16]:
def preprocessing():
    # creating initial dataframe
    X, Y = read_data()
    y_df = pd.DataFrame(Y, columns=['Label'])
    labelencoder = LabelEncoder()
    y_df['Categrory'] = labelencoder.fit_transform(y_df['Label'])


    img_rows, img_cols = X[0].shape


    # transform data set
    if K.image_data_format() == 'channels_first':
        X = X.reshape(X.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        X = X.reshape(X.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
    Y_cat = to_categorical(y_df['Categrory'])
    return X, Y_cat

In [33]:
import tensorflow
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD

class ModelSelection(object):
    def __init__(self):
        pass
    def KFold_GridSearchCV(self, input_dim, X, Y, X_test, y_test, combinations, filename="log.csv", acc_loss_json="hist.json"):
         # create containers for resulting data
         
        
        res_df = pd.DataFrame(columns=['momentum','learning rate','batch size',
                                      'loss1', 'acc1','loss2', 'acc2','loss3', 'acc3'])
        generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)
        hist_dict_global = {}

        for i, combination in enumerate(combinations):
            kf = KFold(n_splits=3, random_state=42, shuffle=False)
            metrics_dict = {}
  
            for j, (train_index, test_index) in enumerate(kf.split(X)):
               if i<10:
                X_train, X_val = X[train_index], X[test_index]
                y_train, y_val = Y[train_index], Y[test_index]
                wresnet_ins = WideResidualNetwork(combination[2], combination[0], in_dim,combination[4], nb_classes=4, N=2, k=2, dropout=0.0)
                model = wresnet_ins.create_wide_residual_network()
                opt = tensorflow.keras.optimizers.SGD(learning_rate=combination[0])
                model.compile(loss='categorical_crossentropy', optimizer=opt, metrics= ['accuracy'])
                hist = model.fit_generator(generator.flow(X_train, y_train, batch_size=combination[1]), steps_per_epoch=len(X_train) // combination[1], epochs=combination[3],
                                    validation_data=(X_val, y_val),
                                    validation_steps=len(X_val) // combination[1],)
                loss, acc = model.evaluate(X_test, y_test)

                metrics_dict[j+1] = {"loss": loss, "acc": acc, "epoch_stopped": combination[3]}
                graph_loss_acc = {"val_loss":hist.history["val_loss"], "train_loss":hist.history["loss"], "epoch_stopped": combination[3], 'learning rate': combination[0],
                                     'batch size': combination[1], 'reg_penalty': combination[2]}
                with open(acc_loss_json, 'w', encoding='utf-8') as f:
                  json.dump(graph_loss_acc, f, ensure_ascii=False, indent=4)

            if i <10:
              row = {'momentum': combination[4],'learning rate': combination[0],
                        'batch size': combination[1], 'reg_penalty': combination[2],
                        'epoch_stopped1': metrics_dict[1]["epoch_stopped"], 
                        'loss1': metrics_dict[1]["loss"],'acc1': metrics_dict[1]["acc"],
                        'acc1': metrics_dict[1]["acc"],'epoch_stopped2': metrics_dict[2]["epoch_stopped"],
                        'loss2': metrics_dict[2]["loss"],'acc2': metrics_dict[2]["acc"],
                        'epoch_stopped3': metrics_dict[3]["epoch_stopped"],
                        'loss3': metrics_dict[3]["loss"], 'acc3': metrics_dict[3]["acc"]}
              res_df = res_df.append(row , ignore_index=True)
              res_df.to_csv(filename, sep=";")





In [ ]:
if __name__ == "__main__":
    learning_rate = [0.1, 0.01]
    batch_size = [64,128,256]
    reg_penalty = [0.01, 0.001, 0.0001]
    epochs = [50,100,150]
    momentum = [0.9]
    in_dim = (32,32,1)
    grid_result = "grid_16_9.csv"
    acc_loss_json = "history.json"

    # create list of all different parameter combinations
    param_grid = dict(learning_rate = learning_rate, batch_size = batch_size, 
                      reg_penalty = reg_penalty, epochs = epochs, momentum=momentum)
    combinations = list(product(*param_grid.values()))
    X, Y = preprocessing()
    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.1, shuffle=False)
    print(combinations)
    instance  = ModelSelection()
    instance.KFold_GridSearchCV(in_dim,X_train,y_train,X_test, y_test, combinations, grid_result, acc_loss_json)

[(0.1, 64, 0.01, 50, 0.9), (0.1, 64, 0.01, 100, 0.9), (0.1, 64, 0.01, 150, 0.9), (0.1, 64, 0.001, 50, 0.9), (0.1, 64, 0.001, 100, 0.9), (0.1, 64, 0.001, 150, 0.9), (0.1, 64, 0.0001, 50, 0.9), (0.1, 64, 0.0001, 100, 0.9), (0.1, 64, 0.0001, 150, 0.9), (0.1, 128, 0.01, 50, 0.9), (0.1, 128, 0.01, 100, 0.9), (0.1, 128, 0.01, 150, 0.9), (0.1, 128, 0.001, 50, 0.9), (0.1, 128, 0.001, 100, 0.9), (0.1, 128, 0.001, 150, 0.9), (0.1, 128, 0.0001, 50, 0.9), (0.1, 128, 0.0001, 100, 0.9), (0.1, 128, 0.0001, 150, 0.9), (0.1, 256, 0.01, 50, 0.9), (0.1, 256, 0.01, 100, 0.9), (0.1, 256, 0.01, 150, 0.9), (0.1, 256, 0.001, 50, 0.9), (0.1, 256, 0.001, 100, 0.9), (0.1, 256, 0.001, 150, 0.9), (0.1, 256, 0.0001, 50, 0.9), (0.1, 256, 0.0001, 100, 0.9), (0.1, 256, 0.0001, 150, 0.9), (0.01, 64, 0.01, 50, 0.9), (0.01, 64, 0.01, 100, 0.9), (0.01, 64, 0.01, 150, 0.9), (0.01, 64, 0.001, 50, 0.9), (0.01, 64, 0.001, 100, 0.9), (0.01, 64, 0.001, 150, 0.9), (0.01, 64, 0.0001, 50, 0.9), (0.01, 64, 0.0001, 100, 0.9), (0.01,